## Cobot Projectile Obstacle Avoidance
Work Items:
1. Obstacle generator
2. YOLO image segmentation
3. Cobot control and DQN

In [1]:
import pybullet as p
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import cobot_ai4robotics

In [ ]:
# Basic code to open the GUI environment window.
# https://www.gymlibrary.dev/content/environment_creation/ - make a custom gym environment.

env = gym.make("cobot_ai4robotics", apply_api_compatibility=True, renders=False, isDiscrete=False)
env.reset()

# Modify joint control so that it is a change to the current joint position, not specifying a joint position.
origin = np.array([0,0,0,0,0,0,0])
while True:
    ob, reward, done, _, info = env.step(env.action_space.sample()) # Things will not move unless you call env.step.
    if done:
        break
    # print(ob)
    # print(env.action_space.sample())
    # First 7 items in ob are the pose (or they're meant to be, they seem dodgy?). The next 4 numbers are the bounding box coordinates compressed to be from 0-1, and the last item is the distance to camera.
env.close()

error: GetBasePositionAndOrientation failed.

In [ ]:
env.close()